In [1]:
import requests
import pandas as pd
from datetime import datetime, timezone

In [3]:
def get_historic(crypto_id="bitcoin", currency="usd", days=365):
    """
    Obtém o histórico de preços da criptomoeda utilizando a API do CoinGecko.

    Parâmetros:
        - crypto_id (str): O ID da criptomoeda (padrão: 'bitcoin').
        - currency (str): A moeda em que a criptomoeda será cotada (padrão: 'usd').
        - days (int): O número de dias históricos a serem obtidos (padrão: 365).

    Retorna:
        - DataFrame: Contendo os dados de preço e data da criptomoeda.
    """
    try:
        # Configurações da URL da API CoinGecko
        url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
        params = {"vs_currency": currency, "days": days}

        # Fazendo a requisição
        response = requests.get(url, params=params)
        response.raise_for_status()  # Levanta exceções para status HTTP 4xx/5xx

        # Processa os dados retornados
        data = response.json()

        # Verifica se a chave 'prices' está presente
        if "prices" not in data:
            raise ValueError("Dados de preços não encontrados na resposta da API.")

        prices = data["prices"]  # Lista de timestamps e preços
        df = pd.DataFrame(prices, columns=["timestamp", "price"])
        df["date"] = pd.to_datetime(df["timestamp"], unit="ms")

        return df
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API do CoinGecko: {e}")
        return None
    except ValueError as e:
        print(f"Erro de dados: {e}")
        return None
    except Exception as e:
        print(f"Erro inesperado: {e}")
        return None

In [9]:
get_historic(days=365)

,timestamp,price,date
0,1710288000000,71467.173503,2024-03-13 00:00:00
1,1710374400000,73097.767027,2024-03-14 00:00:00
2,1710460800000,71420.031801,2024-03-15 00:00:00
3,1710547200000,69497.763193,2024-03-16 00:00:00
4,1710633600000,65292.313293,2024-03-17 00:00:00
...,...,...,...
361,1741478400000,86142.983359,2025-03-09 00:00:00
362,1741564800000,80751.138933,2025-03-10 00:00:00
363,1741651200000,78783.940579,2025-03-11 00:00:00
364,1741737600000,82799.108029,2025-03-12 00:00:00


In [20]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

def get_historic_full(crypto_id="bitcoin", currency="usd", start_date="2010-01-01"):
    """
    Obtém o histórico completo de preços de uma criptomoeda desde uma data específica até hoje.

    Parâmetros:
        - crypto_id (str): O ID da criptomoeda (padrão: 'bitcoin').
        - currency (str): A moeda em que a criptomoeda será cotada (padrão: 'usd').
        - start_date (str): Data inicial no formato 'YYYY-MM-DD' (padrão: '2010-01-01').

    Retorna:
        - DataFrame: Contendo os dados de preço e data da criptomoeda.
    """
    try:
        start_dt = datetime.strptime(start_date, "%Y-%m-%d")
        end_dt = datetime.today()

        all_data = []
        current_dt = start_dt

        while current_dt < end_dt:
            next_dt = min(current_dt + timedelta(days=365), end_dt)
            days = (next_dt - current_dt).days

            print(f"🔄 Obtendo dados de {current_dt.date()} até {next_dt.date()}")

            url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart"
            params = {"vs_currency": currency, "days": days}

            for attempt in range(3):  # Tenta no máximo 3 vezes
                try:
                    response = requests.get(url, params=params)
                    if response.status_code == 429:
                        print("⚠️ Rate limit excedido. Aguardando 20 segundos...")
                        time.sleep(20)
                        continue  # Tenta novamente

                    response.raise_for_status()
                    break  # Sai do loop se a requisição for bem-sucedida
                except requests.exceptions.RequestException as e:
                    print(f"Erro na requisição ({attempt+1}/3): {e}")
                    time.sleep(10)

            data = response.json()
            if "prices" not in data:
                print("❌ Nenhum dado de preço retornado. Pulando para o próximo período...")
                current_dt = next_dt
                continue  # Pula para o próximo intervalo

            df = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
            df["date"] = pd.to_datetime(df["timestamp"], unit="ms")

            all_data.append(df)
            current_dt = next_dt

            # Espera antes da próxima requisição para evitar bloqueio
            time.sleep(10)

        final_df = pd.concat(all_data).drop_duplicates().reset_index(drop=True)
        return final_df

    except Exception as e:
        print(f"🚨 Erro inesperado: {e}")
        return None

# Obtendo histórico desde 2010 com tratamento de rate limit
df_crypto = get_historic_full()
if df_crypto is not None:
    print(df_crypto.head())


🔄 Obtendo dados de 2010-01-01 até 2011-01-01
🔄 Obtendo dados de 2011-01-01 até 2012-01-01
🔄 Obtendo dados de 2012-01-01 até 2012-12-31
🔄 Obtendo dados de 2012-12-31 até 2013-12-31
🔄 Obtendo dados de 2013-12-31 até 2014-12-31
🔄 Obtendo dados de 2014-12-31 até 2015-12-31
🔄 Obtendo dados de 2015-12-31 até 2016-12-30
🔄 Obtendo dados de 2016-12-30 até 2017-12-30
⚠️ Rate limit excedido. Aguardando 20 segundos...
⚠️ Rate limit excedido. Aguardando 20 segundos...
⚠️ Rate limit excedido. Aguardando 20 segundos...
❌ Nenhum dado de preço retornado. Pulando para o próximo período...
🔄 Obtendo dados de 2017-12-30 até 2018-12-30
🔄 Obtendo dados de 2018-12-30 até 2019-12-30
🔄 Obtendo dados de 2019-12-30 até 2020-12-29
🔄 Obtendo dados de 2020-12-29 até 2021-12-29
🔄 Obtendo dados de 2021-12-29 até 2022-12-29
🔄 Obtendo dados de 2022-12-29 até 2023-12-29
🔄 Obtendo dados de 2023-12-29 até 2024-12-28
⚠️ Rate limit excedido. Aguardando 20 segundos...
⚠️ Rate limit excedido. Aguardando 20 segundos...
⚠️ Rate

In [22]:
df_crypto.to_parquet('ibov_20250312.parquet')

In [28]:
df_crypto['date'].min()

Timestamp('2024-03-13 00:00:00')

In [1]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

def get_historic_range(crypto_id="bitcoin", currency="usd", start_date="2010-01-01", end_date="2025-03-12"):
    """
    Obtém o histórico completo de preços de uma criptomoeda em períodos menores para evitar rate limits.

    Parâmetros:
        - crypto_id (str): O ID da criptomoeda (padrão: 'bitcoin').
        - currency (str): A moeda em que a criptomoeda será cotada (padrão: 'usd').
        - start_date (str): Data inicial no formato 'YYYY-MM-DD' (padrão: '2010-01-01').
        - end_date (str): Data final no formato 'YYYY-MM-DD' (padrão: '2025-03-12').

    Retorna:
        - DataFrame: Contendo os dados de preço diário da criptomoeda.
    """
    base_url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart/range"
    all_data = []

    # Converte strings para timestamps (em segundos)
    start_timestamp = int(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
    end_timestamp = int(datetime.strptime(end_date, "%Y-%m-%d").timestamp())

    # Dividindo a requisição em períodos menores (1 ano por vez)
    current_timestamp = start_timestamp
    while current_timestamp < end_timestamp:
        next_timestamp = min(current_timestamp + 365 * 24 * 60 * 60, end_timestamp)
        params = {
            "vs_currency": currency,
            "from": current_timestamp,
            "to": next_timestamp
        }

        print(f"🔄 Obtendo dados de {datetime.utcfromtimestamp(current_timestamp).strftime('%Y-%m-%d')} "
              f"até {datetime.utcfromtimestamp(next_timestamp).strftime('%Y-%m-%d')}...")

        try:
            response = requests.get(base_url, params=params)
            if response.status_code == 429:
                print("⚠️ Rate limit excedido. Aguardando 30 segundos...")
                time.sleep(30)
                response = requests.get(base_url, params=params)

            response.raise_for_status()
            data = response.json()

            if "prices" in data:
                all_data.extend(data["prices"])
            else:
                print(f"⚠️ Nenhum dado encontrado para {params}")

        except requests.exceptions.RequestException as e:
            print(f"🚨 Erro ao acessar a API do CoinGecko: {e}")
            return None

        # Evita sobrecarregar a API (pausa de 10 segundos entre requisições)
        time.sleep(10)
        current_timestamp = next_timestamp

    # Converte os dados para um DataFrame
    df = pd.DataFrame(all_data, columns=["timestamp", "price"])
    df["date"] = pd.to_datetime(df["timestamp"], unit="ms")

    # Remove duplicatas e mantém apenas um valor por dia
    df = df.groupby(df["date"].dt.date).first().reset_index()

    return df[["date", "price"]]

# Chamando a função para obter os dados desde 2010
df_crypto = get_historic_range()
if df_crypto is not None:
    print(df_crypto.head())
    print(df_crypto.tail())


🔄 Obtendo dados de 2010-01-01 até 2011-01-01...
🚨 Erro ao acessar a API do CoinGecko: 401 Client Error: Unauthorized for url: https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1262304000&to=1293840000


In [2]:
import yfinance as yf

# Obtém os dados históricos do Bitcoin desde 2010
df_crypto = yf.download("BTC-USD", start="2010-01-01", end="2025-03-12")

# Apenas colunas de interesse
df_crypto = df_crypto[["Close"]].reset_index()
df_crypto.columns = ["date", "price"]

print(df_crypto.head(), df_crypto.tail())


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

        date       price
0 2014-09-17  457.334015
1 2014-09-18  424.440002
2 2014-09-19  394.795990
3 2014-09-20  408.903992
4 2014-09-21  398.821014            date         price
3824 2025-03-07  86742.671875
3825 2025-03-08  86154.593750
3826 2025-03-09  80601.039062
3827 2025-03-10  78532.000000
3828 2025-03-11  82862.210938


In [3]:
df_crypto.head()

,date,price
0,2014-09-17,457.334015
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014


In [4]:
df_crypto.tail()

,date,price
3824,2025-03-07,86742.671875
3825,2025-03-08,86154.593750
3826,2025-03-09,80601.039062
3827,2025-03-10,78532.000000
3828,2025-03-11,82862.210938


In [5]:
df_crypto.describe()

,date,price
count,3829,3829.000000
mean,2019-12-14 00:00:00,20982.033193
min,2014-09-17 00:00:00,178.102997
25%,2017-05-01 00:00:00,1421.599976
50%,2019-12-14 00:00:00,9552.860352
75%,2022-07-28 00:00:00,33901.527344
max,2025-03-11 00:00:00,106146.265625
std,NaN,24208.664323


In [6]:
df_crypto.to_parquet('yfinance_20250312.parquet')